<a href="https://colab.research.google.com/github/jiwoong2/deeplearning/blob/main/%EC%9D%B8%EA%B3%B5%EC%8B%A0%EA%B2%BD%EB%A7%9D_%EA%B5%AC%ED%98%842.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys, os
import numpy as np
import pickle # 피클라이브러리는 텍스트파일의 비효율적인 저장방식을 효율적으로 개선
import tensorflow as tf
import matplotlib.pyplot as plt

# 시그모이드, 소프트맥스 함수

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))  # np.exp(x)는 밑이 자연상수 e 이고 x가 지수인 수를 반환한다.

def softmax(x):
    if x.dim == 2:   # x가 행렬일 경우 scalar = 0, vector = 1, matrix = 2, 위의 경우 vector를 처리하므로 후에 배치처리때 사용됨.
        x = x.T
        x = x - np.max(x, axis = 0)
        y = np.exp(x) / np.sum(np.exp(x), axis = 0)
        return y.T

    x = x - np.max(x) # 오버플로를 방지하기위해 작성. 노트참고. , vector에 scalar를 빼면 numpy에서 알아서 원소별로 연산을 한다.
    return np.exp(x) / np.sum(np.exp(x))  # normalize

# Data 전처리

In [ ]:
def get_data():
    Mnist = tf.keras.datasets.mnist
    (x_train, t_train), (x_test, t_test) = Mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0 # normalize -> 많은 경우 피쳐가 여러개이므로 큰 숫자를 가지는 피쳐의 영향력이 너무 커지므로 모든 피쳐값을 0부터 1까지의 값으로 정규화 한다.
    x_train = x_train.reshape(60000, 784) # flatten
    x_test = x_test.reshape(10000, 784)
    return x_test, t_test

# 모델 구현

# 배치처리 구현
배치처리의 자세한 원리는 노트 참고.

예제 모델에서는 배치처리후 시간이 절반 가까이 단축된다.

In [ ]:
x, t = get_data() # 테스트 데이터와 라벨을 x, t 변수에 저장한다.

batch_size = 100 # 한번에 처리할 배치 사이즈를 지정 한다.
accuracy_cnt = 0 # 추론결과 정답갯수를 저장할 변수.

In [ ]:
for i in range(0, len(x), batch_size):
    x_batch = x[i:i+batch_size] # batch size가 100 이므로 인덱스 0부터 99까지의 100개의 이미지를 슬라이싱 한다. 100*784 행렬이 반환 된다.(각 행은 falten한 이미지이다.)
    y_batch = predict(network, x_batch) # 슬라이싱한 100가지 이미지를 모델로 추론해 나온 값을 저장한다. 100*10 행렬이 반환 된다.(각 행은 각 사진의 확률벡터이다.)
    p = np.argmax(y_batch, axis = 1) # argmax 함수는 가장 큰값의 인덱스를 반화하며, axis = 1 은 앞의 작업을 각 행별로 적용한다. 결과적으로 100차원 벡터를 반환 한다.  axis = 0 은 열에대해서 적용한다.
    accuracy_cnt += np.sum(p == t[i:i+batch_size]) # t는 라벨이고 마찬가지로 슬라이싱해 추론결과와 비교한다. bool형의 리스트를 반환하며 true는 1 false는 0 이므로 합하면 정답 갯수를 알 수 있다.

print("Accuracy:"+str(float(accuracy_cnt) / len(x)))

# Error

In [ ]:
error = [] # 모델의 추론이 틀린 인덱스를 저장하기 위한 리스트.

for i in range(len(x)):
    y = predict(network, x[i])
    p = np.argmax(y) # 확률이 가장 높은 원소의 인덱스를 얻는다.
    if p != t[i]:
        error.append(i)

print(error)

plt.figure(figsize = (10, 10))
for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(x[error[i]].reshpae(28, 28), cmap = plt.cm.binary)
    plt.xlabel(t[error[i]])
plt.show()